In [1]:
from configs.config import get_path
import src.utils.gen_jobs as gen_jobs

import pandas as pd
pd.set_option('display.max_rows', 26)

In [2]:
import src.utils.gen_interarrival as gen_ta
import math
import numpy as np

In [3]:
basic_data_path = get_path("data", "basic")
df_instance = pd.read_csv(basic_data_path / "00_instance.csv")
df_instance

,Job,Operation,Machine,Processing Time
0,job 0,0,M0,29
1,job 0,1,M1,78
2,job 0,2,M2,9
3,job 0,3,M3,36
4,job 0,4,M4,49
...,...,...,...,...
95,job 9,5,M9,76
96,job 9,6,M5,47
97,job 9,7,M3,52
98,job 9,8,M4,90


In [4]:
def generate_job_times(df_jssp, start_time: int = 0, u_b_mmax: float = 0.9,
                       shift_length: int = 1440, seed: int = 122) -> pd.DataFrame:
    """
    Berechnet ein DataFrame mit Arrival, Ready Time und Earliest End.

    Parameter:
    - df_jssp: DataFrame mit ['Job', 'Machine', 'Processing Time']
    - start_time: Produktionsstartzeit in Minuten
    - u_b_mmax: Ziel-Auslastung der Engpassmaschine
    - shift_length: Schichtlänge in Minuten
    - seed: Zufalls-Seed für Ankunftszeiten

    Rückgabe:
    - DataFrame mit Spalten ['Job', 'Arrival', 'Ready Time', 'Processing Time', 'Earliest End']
    """
    # 1. Mittelwert der Interarrival-Zeiten berechnen
    t_a = gen_ta.calculate_mean_interarrival_time(df_jssp, u_b_mmax=u_b_mmax)

    # 2. Jobliste und Arrivals generieren
    job_order = df_jssp['Job'].drop_duplicates().tolist()
    arrivals = gen_ta.calculate_arrivals(df_jssp, mean_interarrival_time=t_a, start_time=start_time, random_seed=seed)

    # 3. Gesamte Processing Time pro Job berechnen
    processing_times = df_jssp.groupby('Job')['Processing Time'].sum()

    # 4. DataFrame zusammenbauen
    data = []
    for job, arrival in zip(job_order, arrivals):
        ready_time = math.ceil((arrival + 1) / shift_length) * shift_length
        processing_time = processing_times[job]
        earliest_end = ready_time + processing_time
        data.append({
            'Job': job,
            'Arrival': arrival,
            'Ready Time': ready_time,
            'Processing Time': processing_time,
            'Earliest End': earliest_end
        })

    return pd.DataFrame(data)

In [5]:
df_jobs_times = generate_job_times(df_instance, u_b_mmax = 0.90, shift_length=1440)
df_jobs_times

,Job,Arrival,Ready Time,Processing Time,Earliest End
0,job 0,0,1440,395,1835
1,job 1,84,1440,510,1950
2,job 2,106,1440,568,2008
3,job 3,148,1440,655,2095
4,job 4,185,1440,393,1833
5,job 5,251,1440,496,1936
6,job 6,312,1440,416,1856
7,job 7,329,1440,539,1979
8,job 8,344,1440,597,2037
9,job 9,429,1440,540,1980


### Generierung für mehre Tage (Schichten von 1440 Minuten)

In [6]:
arrival_day_numb = 2

df_jssp, df_jobs_times = gen_jobs.create_jobs_for_shifts(df_instance, shift_count= arrival_day_numb, u_b_mmax= 0.92, 
                                                         shift_length = 1440, shuffle = True, job_seed= 50, arrival_seed = 120)
df_jobs_times

,Job,Arrival,Ready Time,Processing Time,Earliest End
0,Job_000,0,1440,395,1835
1,Job_001,49,1440,510,1950
2,Job_002,116,1440,568,2008
3,Job_003,160,1440,655,2095
4,Job_004,203,1440,393,1833
...,...,...,...,...,...
34,Job_034,2579,2880,597,3477
35,Job_035,2589,2880,496,3376
36,Job_036,2597,2880,393,3273
37,Job_037,2663,2880,539,3419
